In [1]:
import numpy as np
import pandas as pd

import random

import sys
sys.path.append("../")

from src.object import *
from src.create import *

from config.formation import *

In [2]:
def apply_points(row):
    return row.win*3+row.row

In [3]:
df_name = pd.read_csv("../data/csv/NationalNames.csv")
df_name_list = list(df_name[df_name.Gender=='M'].Name.sample(500))

In [4]:
df_place = pd.read_csv("../data/csv/results.csv")
df_place = df_place.city.unique()

In [5]:
class League:
    def __init__(self, name, teams, num, category, relegation_num=0, promotion_num=0):
        self.name = name
        self.teams = teams
        self.num = num
        self.category = category
        self.result = {}
        self.champion = {}
        
        self.relegation = {}
        self.relegation_num = relegation_num
        self.promotion = {}
        self.promotion_num = promotion_num
    
    def play_1year(self, season_name):
        all_team_name = [s.name for s in self.teams]
        output = pd.DataFrame(np.zeros((len(all_team_name), 5)), 
                              index=all_team_name, 
                              columns=["win", "lose", "row", "得点", "失点"], 
                              dtype=np.int8)
        
        # 一年間試合をこなす
        for i in range(20):
            for j in range(20):
                if i!= j:
                    game = Game(home=self.teams[i], away=self.teams[j])
                    game.battle()
                    
                    home_team_name = self.teams[i].name
                    away_team_name = self.teams[j].name
                    
                    if game.result=="home":
                        output.loc[home_team_name, "win"] += 1
                        output.loc[away_team_name, "lose"] += 1
                    elif game.result=="away":
                        output.loc[away_team_name, "win"] += 1
                        output.loc[home_team_name, "lose"] += 1
                    else:
                        output.loc[home_team_name, "row"] += 1
                        output.loc[away_team_name, "row"] += 1

                    output.loc[home_team_name, "得点"] += game.home_goal
                    output.loc[home_team_name, "失点"] += game.away_goal

                    output.loc[away_team_name, "得点"] += game.away_goal
                    output.loc[away_team_name, "失点"] += game.home_goal
        
        output["得失点差"] = output["得点"]-output["失点"]
        output["Points"] = output.apply(apply_points, axis=1)
        output = output.sort_values("Points", ascending=False)
        output["順位"] = [f"{i}位" for i in range(1, 21)]
        output["リーグ名"] = [f"{self.name}" for i in range(20)]
        
        for team in self.teams:
            team.result.loc[season_name] = output.loc[team.name, :]
        
        if list(output.index)[0] in self.champion.keys():
            self.champion[list(output.index)[0]].append(season_name)
        else:
            self.champion[list(output.index)[0]] = [season_name]
        
        self.result[season_name] = output
        
        # 昇格決定
        if self.category!="top":
            self.promotion[season_name] = list(self.result[season_name][:self.promotion_num].index)
        
        #降格決定
        if self.category!="lowest":
            self.relegation[season_name] = list(self.result[season_name][-self.relegation_num:].index)

In [6]:
class ProSoccerLeague:
    def __init__(self, name, leagues):
        self.name = name
        self.leagues = leagues
    
    def play_1year(self, season_name):
        for league in self.leagues:
            league.play_1year(season_name)
        
        for index in range(len(self.leagues)):
            if self.leagues[index].category!="top":
                promotion = self.leagues[index].promotion[season_name]
                promotion_team = [t for t in self.leagues[index].teams if t.name in promotion]
                self.leagues[index].teams = [s for s in self.leagues[index].teams if s not in promotion_team]
                self.leagues[index-1].teams.extend(promotion_team)

            if self.leagues[index].category!="lowest":
                relegation = self.leagues[index].relegation[season_name]
                relegation_team = [t for t in self.leagues[index].teams if t.name in relegation]
                self.leagues[index].teams = [s for s in self.leagues[index].teams if s not in relegation_team]
                self.leagues[index+1].teams.extend(relegation_team)

In [16]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(75, 80)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i], 
             formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L1 = League(name="premia", 
            teams=league_team,
            num=20,
            category='top',
            relegation_num=3)

In [17]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(67, 77)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+20], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L2 = League(name="championship",
            teams=league_team,
            num=20,
            category='middle',
            relegation_num=3,
            promotion_num=3)

In [18]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(60, 70)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+40], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L3 = League(name="EFL1", 
            teams=league_team,
            num=20,
            category='lowest',
            relegation_num=0,
            promotion_num=3)

In [19]:
England = ProSoccerLeague(name="england", leagues=[L1, L2, L3])
for year in range(2000, 2020):
    print(year)
    England.play_1year(f"{year}")

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [21]:
England.leagues[0].champion

{'Stoke-on-Trent': ['2000',
  '2002',
  '2003',
  '2004',
  '2007',
  '2008',
  '2010',
  '2011',
  '2016'],
 'Shrewsbury': ['2001',
  '2005',
  '2012',
  '2013',
  '2014',
  '2015',
  '2017',
  '2018',
  '2019'],
 'Newark': ['2006', '2009']}

In [12]:
England.leagues[0].result['2001']

,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
Bangor,28,2,8,62,15,47,92,1位,premia
Blackburn,27,3,8,72,20,52,89,2位,premia
Sheffield,26,5,7,64,19,45,85,3位,premia
Birmingham,23,9,6,47,19,28,75,4位,premia
Belfast,19,5,14,42,19,23,71,5位,premia
Dundee,19,11,8,38,31,7,65,6位,premia
Sunderland,20,15,3,54,33,21,63,7位,premia
Stoke-on-Trent,16,14,8,48,32,16,56,8位,premia
Shrewsbury,15,13,10,31,32,-1,55,9位,premia
Edinburgh,14,12,12,33,30,3,54,10位,premia


In [33]:
[t.name for t in England.leagues[2].teams]

['Middlesbrough',
 'Antwerp',
 'Rotterdam',
 'Prague',
 'Haarlem',
 'Geneva',
 'Gothenburg',
 'The Hague',
 'Bradford',
 'Gentilly',
 'Milan',
 'Viña del Mar',
 'Kristiania',
 'Llandudno',
 'Motherwell',
 'Duisburg',
 'Basel',
 'Aberdeen',
 'Nottingham',
 'Cardiff']

In [34]:
num = 0
print(England.leagues[2].teams[num].name)
England.leagues[2].teams[num].result

Middlesbrough


,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
2000,3,29,6,31,137,-106,15,19位,EFL1
2001,6,25,7,32,111,-79,25,17位,EFL1
2002,6,28,4,30,107,-77,22,17位,EFL1
2003,6,28,4,44,126,-82,22,16位,EFL1
2004,7,30,1,39,116,-77,22,17位,EFL1
2005,7,27,4,36,112,-76,25,16位,EFL1
2006,3,30,5,34,139,-105,14,19位,EFL1
2007,5,24,9,39,106,-67,24,17位,EFL1
2008,7,27,4,39,131,-92,25,16位,EFL1
2009,7,26,5,33,103,-70,26,16位,EFL1


In [20]:
England.leagues[0].relegation

{'2000': ['Richmond', 'Crewe', 'Paisley'],
 '2001': ['Edinburgh', 'Montevideo', 'Glasgow'],
 '2002': ['Richmond', 'Birmingham', 'Crewe'],
 '2003': ['Budapest', 'Paris', 'Liverpool'],
 '2004': ['Montevideo', 'Portsmouth', 'Newcastle'],
 '2005': ['Birmingham', 'Edinburgh', 'Bristol'],
 '2006': ['Montevideo', 'Crewe', 'Portsmouth'],
 '2007': ['Edinburgh', 'Budapest', 'Paris'],
 '2008': ['Montevideo', 'Portsmouth', 'Richmond'],
 '2009': ['Southampton', 'Paris', 'Liverpool'],
 '2010': ['Birmingham', 'Budapest', 'Richmond'],
 '2011': ['Montevideo', 'Paris', 'Crewe'],
 '2012': ['Portsmouth', 'Budapest', 'Richmond'],
 '2013': ['Edinburgh', 'Montevideo', 'Crewe'],
 '2014': ['Wrexham', 'Richmond', 'Paris'],
 '2015': ['Montevideo', 'Birmingham', 'Portsmouth'],
 '2016': ['Budapest', 'Paris', 'Bristol'],
 '2017': ['Liverpool', 'Birmingham', 'Portsmouth'],
 '2018': ['Edinburgh', 'Richmond', 'Glasgow'],
 '2019': ['Montevideo', 'Portsmouth', 'Budapest']}

In [26]:
England.leagues[1].promotion, England.leagues[1].relegation

({'2000': ['Southampton', 'Dundee', 'Montevideo'],
  '2001': ['Budapest', 'Crewe', 'Richmond'],
  '2002': ['Paris', 'Edinburgh', 'Montevideo'],
  '2003': ['Portsmouth', 'Birmingham', 'Newcastle'],
  '2004': ['Liverpool', 'Budapest', 'Bristol'],
  '2005': ['Crewe', 'Montevideo', 'Portsmouth'],
  '2006': ['Birmingham', 'Edinburgh', 'Paris'],
  '2007': ['Richmond', 'Montevideo', 'Portsmouth'],
  '2008': ['Edinburgh', 'Paris', 'Budapest'],
  '2009': ['Portsmouth', 'Montevideo', 'Richmond'],
  '2010': ['Southampton', 'Crewe', 'Paris'],
  '2011': ['Liverpool', 'Budapest', 'Richmond'],
  '2012': ['Paris', 'Montevideo', 'Crewe'],
  '2013': ['Birmingham', 'Richmond', 'Budapest'],
  '2014': ['Edinburgh', 'Portsmouth', 'Montevideo'],
  '2015': ['Wrexham', 'Paris', 'Bristol'],
  '2016': ['Montevideo', 'Birmingham', 'Portsmouth'],
  '2017': ['Budapest', 'Glasgow', 'Richmond'],
  '2018': ['Liverpool', 'Birmingham', 'Portsmouth'],
  '2019': ['Edinburgh', 'Glasgow', 'Bristol']},
 {'2000': ['Aberdeen',

In [32]:
England.leagues[2].result['2019']

,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
Colombes,32,4,2,110,11,99,98,1位,EFL1
Arnhem,30,4,4,126,30,96,94,2位,EFL1
Greenock,29,4,5,107,16,91,92,3位,EFL1
Basel,29,4,5,118,24,94,92,4位,EFL1
Duisburg,25,7,6,97,32,65,81,5位,EFL1
Gothenburg,24,7,7,101,34,67,79,6位,EFL1
Geneva,23,7,8,89,31,58,77,7位,EFL1
Llandudno,23,13,2,89,58,31,71,8位,EFL1
Rotterdam,23,13,2,97,50,47,71,9位,EFL1
Bradford,14,18,6,55,61,-6,48,10位,EFL1
